In [ ]:
import numpy             as np
import matplotlib.pyplot as plt
import scipy.sparse      as sparse
import scipy.special     as sp
import hylife.utilitis_FEEC.bsplines             as bsp
import hylife.utilitis_FEEC.evaluation           as eva
import hylife.geometry.mappings_analytical       as mapping
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D

In [ ]:
Nel       = [16,16, 4]              # mesh generation on logical domain
bc        = [True, True, True]      # boundary conditions (True: periodic, False: else)
p         = [3, 3, 2]               # spline degrees  


el_b      = [np.linspace(0., 1., Nel + 1) for Nel in Nel]                      # element boundaries
delta     = [1/Nel for Nel in Nel]                                             # element sizes
T         = [bsp.make_knots(el_b, p, bc) for el_b, p, bc in zip(el_b, p, bc)]  # knot vectors (for N functions)
t         = [T[1:-1] for T in T]                                               # reduced knot vectors (for D function)
NbaseN    = [Nel + p - bc*p for Nel, p, bc in zip(Nel, p, bc)]                 # number of basis functions (N functions)
NbaseD    = [NbaseN - (1 - bc) for NbaseN, bc in zip(NbaseN, bc)]              #number of basis functions (D functions)

kind_map = 1               #slab geometry
L_map    = [4., 2., 1.] #length of the simulation box 

args_map = kind_map, L_map #putting it together

In [ ]:
import hylife.utilitis_FEEC.projectors_global    as proj_glob
import hylife.utilitis_FEEC.projectors_local     as proj_loc
import hylife.utilitis_FEEC.derivatives          as der

proj_glob = proj_glob.projectors_3d(T, p, bc)
proj_loc  = proj_loc.projectors_local_3d(T, p, bc)

proj_glob.assemble_V3()
proj_glob.assemble_V2()
proj_glob.assemble_V1()
derivatives = der.discrete_derivatives(T, p, bc)
DIV  = derivatives.DIV_3d()

#### Definition of the initialization functions

In [ ]:
rho_ini_phys = lambda x, y, z : (1+0.5*np.sin(2*np.pi*(x/L_map[0]-y/L_map[1])))
#rho_ini_phys = lambda x, y, z : (np.sin(2*np.pi*x))

In [ ]:
vx_ini_phys = lambda x, y, z : 1.
vy_ini_phys = lambda x, y, z : -0.5
vz_ini_phys = lambda x, y, z : 0.

In [ ]:
#pointwise function evaluation
def rho_ini_pw(xi1, xi2, xi3): 
    x = mapping.f(xi1, xi2, xi3, *args_map, 1) #Analytic Mapping to slab Geometry
    y = mapping.f(xi1, xi2, xi3, *args_map, 2)
    z = mapping.f(xi1, xi2, xi3, *args_map, 3)
    rho = rho_ini_phys(x, y, z) * mapping.det_df(xi1, xi2, xi3, *args_map)
    return rho

In [ ]:
# input, for projector V3:  three 1d point arrays
def rho_ini(xi1_arr, xi2_arr, xi3_arr):
    n1 = xi1_arr.shape[0]
    n2 = xi2_arr.shape[0]
    n3 = xi3_arr.shape[0]
    rho = np.empty((n1, n2, n3), dtype=float) # Returns numpy array of shape [n1,n2,n3] same fct as np.zeros
    for i1 in range(n1):
        for i2 in range(n2):
            for i3 in range(n3):
                idx = i1, i2, i3
                rho[idx] = rho_ini_pw(xi1_arr[i1], xi2_arr[i2], xi3_arr[i3])
    return rho

In [ ]:
#pointwise function evaluation, for velocity as a 1-form
def vk_ini_pw(xi1, xi2, xi3,k):
    x = mapping.f(xi1, xi2, xi3, *args_map, 1)
    y = mapping.f(xi1, xi2, xi3, *args_map, 2)
    z = mapping.f(xi1, xi2, xi3, *args_map, 3)

    df_inv_1k = mapping.df_inv(xi1, xi2, xi3, *args_map, 10+k)
    df_inv_2k = mapping.df_inv(xi1, xi2, xi3, *args_map, 20+k)
    df_inv_3k = mapping.df_inv(xi1, xi2, xi3, *args_map, 30+k)
    
    v =(vx_ini_phys(x, y, z) * df_inv_1k + 
        vy_ini_phys(x, y, z) * df_inv_2k + 
        vz_ini_phys(x, y, z) * df_inv_3k   )
    #f_1k = mapping.df(xi1, xi2, xi3, *args_map, 10+k)
    #f_2k = mapping.df(xi1, xi2, xi3, *args_map, 20+k)
    #f_3k = mapping.df(xi1, xi2, xi3, *args_map, 30+k)
    #
    #v  =(vx_ini_phys(x, y, z) * df_1k + 
    #     vy_ini_phys(x, y, z) * df_2k + 
    #     vz_ini_phys(x, y, z) * df_3k   )
    return v


# input, for projector V1:  two point values and one 1d point array
def v1_ini(xi1_arr, xi2, xi3):
    n1 = xi1_arr.shape[0]
    v1 = np.empty((n1), dtype=float)
    for i1 in range(n1):
            
        v1[i1] = vk_ini_pw(xi1_arr[i1],xi2,xi3,1)
            
    return v1

def v2_ini(xi1, xi2_arr, xi3):
    n2 = xi2_arr.shape[0]
    v2 = np.empty((n2), dtype=float)
    for i2 in range(n2):
            
        v2[i2] = vk_ini_pw(xi1,xi2_arr[i2],xi3,2)
            
    return v2

def v3_ini(xi1, xi2, xi3_arr):
    n3 = xi3_arr.shape[0]
    v3 = np.empty((n3), dtype=float)
    for i3 in range(n3):
            
        v3[i3] = vk_ini_pw(xi1,xi2,xi3_arr[i3],3)
            
    return v3

In [ ]:
v1_coeff,v2_coeff,v3_coeff= proj_glob.PI_1([v1_ini,v2_ini,v3_ini])

In [ ]:
# input, for projector V2:  one point value and two 1d point arrays
def v1_times_rho(xi1, xi2_arr, xi3_arr):
    global rho_coeff
    global v1_coeff
    global v2_coeff
    global v3_coeff
    n2 = xi2_arr.shape[0]
    n3 = xi3_arr.shape[0]
    fun = np.empty((n2, n3), dtype=float)
    
    rho_h = eva.FEM_field_V3_3d(rho_coeff, [np.array([xi1]), xi2_arr, xi3_arr], T, p, bc)
    v1_h,v2_h,v3_h = eva.FEM_field_V1_3d([v1_coeff, v2_coeff, v3_coeff], \
                                           [np.array([xi1]), xi2_arr, xi3_arr], T, p, bc)

    fun = rho_h.reshape(n2,n3)*v1_h.reshape(n2,n3)
    
    return fun
                                            
def v2_times_rho(xi1_arr, xi2, xi3_arr):
    global rho_coeff
    global v1_coeff
    global v2_coeff
    global v3_coeff
    n1 = xi1_arr.shape[0]
    n3 = xi3_arr.shape[0]
    fun = np.empty((n1, n3), dtype=float)
    
    rho_h = eva.FEM_field_V3_3d(rho_coeff, [xi1_arr, np.array([xi2]), xi3_arr], T, p, bc)
    v1_h,v2_h,v3_h = eva.FEM_field_V1_3d([v1_coeff, v2_coeff, v3_coeff], \
                                           [xi1_arr, np.array([xi2]), xi3_arr], T, p, bc)

    fun = rho_h.reshape(n1,n3)*v2_h.reshape(n1,n3)
    
    return fun

                                            
def v3_times_rho(xi1_arr, xi2_arr, xi3):
    global rho_coeff
    global v1_coeff
    global v2_coeff
    global v3_coeff
    n1 = xi1_arr.shape[0]
    n2 = xi2_arr.shape[0]
    fun = np.empty((n1, n2), dtype=float)
    
    rho_h = eva.FEM_field_V3_3d(rho_coeff, [xi1_arr, xi2_arr, np.array([xi3])], T, p, bc)
    v1_h,v2_h,v3_h = eva.FEM_field_V1_3d([v1_coeff, v2_coeff, v3_coeff], \
                                           [xi1_arr, xi2_arr, np.array([xi3])], T, p, bc)

    fun = rho_h.reshape(n1,n2)*v3_h.reshape(n1,n2)
    
    return fun
                                            

#### $\rho_0$ Initialization and Visualization

In [ ]:
rho_coeff_init = proj_glob.PI_3(rho_ini)
rho_coeff = rho_coeff_init

In [ ]:
v1_coeff,v2_coeff,v3_coeff= proj_glob.PI_1([v1_ini,v2_ini,v3_ini])

In [ ]:
def eva_rho_t():
    global rho_coeff
    global v1_coeff
    global v2_coeff
    global v3_coeff
    rhov1_coeff,rhov2_coeff, rhov3_coeff = proj_glob.PI_2([v1_times_rho, v2_times_rho, v3_times_rho])
    rho_t = -(DIV.dot(np.concatenate((rhov1_coeff.flatten(), rhov2_coeff.flatten(), rhov3_coeff.flatten())))).reshape(NbaseD[0], NbaseD[1], NbaseD[2]) 
    return rho_t

#rho_t_test = eva_rho_t()

In [ ]:
# prepare visualization
np1=100
np2=100
np3=1
xi1=np.linspace(0,1,np1)
xi2=np.linspace(0,1,np2)
xi3=np.array([0])

#x_m,y_m,z_m = push_forward_meshgrid([xi1,xi2,xi3]) 
x_m=np.empty((np1,np2,np3))
y_m=np.empty((np1,np2,np3))
z_m=np.empty((np1,np2,np3))
det=np.empty((np1,np2,np3))

for i1 in range(np1):
    for i2 in range(np2):
        for i3 in range(np3):
            idx=i1,i2,i3
            x_m[idx]= mapping.f(xi1[i1],xi2[i2],xi3[i3], *args_map, 1)
            y_m[idx]= mapping.f(xi1[i1],xi2[i2],xi3[i3], *args_map, 2)
            z_m[idx]= mapping.f(xi1[i1],xi2[i2],xi3[i3], *args_map, 2)
            
            det[idx] = mapping.det_df(xi1[i1],xi2[i2],xi3[i3], *args_map)
            
  

In [ ]:
%matplotlib notebook

rho_0_plot = eva.FEM_field_V3_3d(rho_coeff_init, [xi1,xi2,xi3], T, p, bc) /det

#rho_phys_0_plot =push_forward_V3(rho_0_plot,[xi1,xi2,xi3]) #divide by jacobian determinant
    
#x_m, y_m=np.meshgrid(x,y)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(x_m.reshape(np1,np2),y_m.reshape(np1,np2),rho_0_plot.reshape(np1,np2))
rho_ex_0=np.empty((np1,np2,np3))
for i1 in range(np1):
    for i2 in range(np2):
        for i3 in range(np3):
            idx=i1,i2,i3
            rho_ex_0[idx] = rho_ini_phys(x_m[idx],y_m[idx],z_m[idx])
          
ax.plot_surface(x_m.reshape(np1,np2),y_m.reshape(np1,np2),rho_ex_0.reshape(np1,np2))

In [ ]:
#L1error
np.amax(np.abs(rho_ex_0-rho_0_plot))


In [ ]:
# Explicit Euler Integration
rho_coeff = rho_coeff_init
dt=0.01
ntsteps=10
for it in range(ntsteps):
    rho_t=eva_rho_t()
    rho_coeff=(rho_coeff+dt*rho_t)


In [ ]:
rho_1_plot=eva.FEM_field_V3_3d(rho_coeff, [xi1,xi2,xi3], T, p, bc)/det
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(x_m.reshape(np1,np2),y_m.reshape(np1,np2),rho_1_plot.reshape(np1,np2))
#exact density at time t=ntsteps*dt, translated with a constant velocity field
rho_ex_1=np.empty((np1,np2,np3))
for i1 in range(np1):
    for i2 in range(np2):
        for i3 in range(np3):
            idx=i1,i2,i3
            xt = x_m[idx] -ntsteps*dt*vx_ini_phys(x_m[idx],y_m[idx],z_m[idx])
            yt = y_m[idx] -ntsteps*dt*vy_ini_phys(x_m[idx],y_m[idx],z_m[idx])
            zt = z_m[idx] -ntsteps*dt*vz_ini_phys(x_m[idx],y_m[idx],z_m[idx])
            rho_ex_1[idx] = rho_ini_phys(xt,yt,zt)
ax.plot_surface(x_m.reshape(np1,np2),y_m.reshape(np1,np2),rho_ex_1.reshape(np1,np2))

In [ ]:
#L1error
np.amax(np.abs(rho_ex_1-rho_1_plot))